In [95]:
import pandas as pd
import numpy as np
from stats import mode
import tqdm
import warnings
import yaml
import math

import os

warnings.filterwarnings('ignore')

# Описание задачи

Целью данного проекта является разработка сервиса для модели, которая предсказывает пол человека по его HTTP cookies

**В данном jupyter-ноутбуке мы рассмотрим и предобработаем 1 из 10 файлов с данными. После чего аггрегируем отдельно каждый файл в цикле и соединим в 1 датафрейм**

## О соревновании и данных

https://ods.ai/competitions/mtsmlcup

Задача соревнования
- Определение пола и возраста владельца HTTP cookie по истории активности пользователя в интернете на основе синтетических данных.

Метрики соревнования:
* ROC-AUC – для определения пола

Описание колонок файла с данными:
* 'region_name' – Регион
* 'city_name' – Населенный пункт
* 'cpe_manufacturer_name' – Производитель устройства
* 'cpe_model_name' – Модель устройства
* 'url_host' – Домен, с которого пришел рекламный запрос
* 'cpe_type_cd' – Тип устройства (смартфон или что-то другое)
* 'Cpe_model_os_type' – Операционка на устройстве
* 'price' – Оценка цены устройства
* 'date' – Дата
* 'part_of_day' – Время дня (утро, вечер, и тд)
* 'request_cnt' – Число запросов одного пользователя за время дня (поле part_of_day)
* 'user_id' – ID пользователя

Описание колонок файла с таргетами:

* 'age' – Возраст пользователя
* 'is_male' – Признак пользователя : мужчина (1-Да, 0-Нет)
* 'user_id' – ID пользователя

# Data preparing

In [154]:
config_path = '../config/parameters.yaml'
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

preprocessing = config['preprocessing']

In [147]:
preprocessing

{'raw_data_extension': '.parquet',
 'change_col_types': {'region_name': 'category',
  'city_name': 'category',
  'cpe_manufacturer_name': 'category',
  'cpe_model_name': 'category',
  'url_host': 'category',
  'cpe_type_cd': 'category',
  'cpe_model_os_type': 'category',
  'price': 'float32',
  'part_of_day': 'category',
  'request_cnt': 'int8',
  'user_id': 'int32'},
 'columns_fill_na': {'price': -999},
 'replace_values': {'cpe_model_os_type': {'Apple iOS': 'iOS'},
  'cpe_manufacturer_name': {'Huawei Device Company Limited': 'Huawei',
   'Motorola Mobility LLC, a Lenovo Company': 'Motorola',
   'Sony Mobile Communications Inc.': 'Sony',
   'Highscreen Limited': 'Highscreen',
   'Realme Chongqing Mobile Telecommunications Corp Ltd': 'Realme',
   'Realme Mobile Telecommunications (Shenzhen) Co Ltd': 'Realme'}},
 'columns_save_min_max': ['region_cnt',
  'city_cnt',
  'url_host_cnt',
  'part_of_day_morning',
  'part_of_day_day',
  'part_of_day_evening',
  'part_of_day_night',
  'act_days'

In [62]:
df = pd.read_parquet(
    '../data/raw/part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
    engine='fastparquet')

In [63]:
df.shape

(32638709, 12)

In [64]:
df.head(3)

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,20368.0,2022-06-15,morning,1,45098
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,20368.0,2022-06-19,morning,1,45098
2,Краснодарский край,Краснодар,Apple,iPhone 7,avatars.mds.yandex.net,smartphone,iOS,20368.0,2022-06-12,day,1,45098


In [152]:
# для примера сохраним сырые данные для id, по которым нужно сделать предсказание
submit_id = pd.read_parquet(preprocessing['submit_path'])
submit_id.head(3)

,user_id
221301,221301
31271,31271
211594,211594


In [155]:
submit_data = submit_id.merge(df, on='user_id')
submit_data.head(1)

,user_id,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt
0,211594,Ростовская область,Сальск,Xiaomi,Poco X3 Pro,googleads.g.doubleclick.net,smartphone,Android,23876.0,2021-06-22,evening,1


In [157]:
config['evaluate']['submit_data']

'../data/check/submit_data.csv'

In [160]:
# сохраняем данные для примера
submit_data.to_csv(config['evaluate']['submit_data'], index=False)

In [65]:
def change_cols_type(data: pd.DataFrame, col_types_dict: dict) -> pd.DataFrame:
    """
    Изменение типа столбцов на заданные
    :param data: датафрейм
    :param col_types_dict: словарь с признаками и типами данных
    :return: датафрейм
    """
    return data.astype(col_types_dict)

In [66]:
# изменения типа столбцов
df = change_cols_type(df, preprocessing['change_col_types'])

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32638709 entries, 0 to 32638708
Data columns (total 12 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   region_name            category      
 1   city_name              category      
 2   cpe_manufacturer_name  category      
 3   cpe_model_name         category      
 4   url_host               category      
 5   cpe_type_cd            category      
 6   cpe_model_os_type      category      
 7   price                  float32       
 8   date                   datetime64[ns]
 9   part_of_day            category      
 10  request_cnt            int8          
 11  user_id                int32         
dtypes: category(8), datetime64[ns](1), float32(1), int32(1), int8(1)
memory usage: 936.5 MB


In [68]:
df.isna().sum()

region_name                   0
city_name                     0
cpe_manufacturer_name         0
cpe_model_name                0
url_host                      0
cpe_type_cd                   0
cpe_model_os_type             0
price                    674264
date                          0
part_of_day                   0
request_cnt                   0
user_id                       0
dtype: int64

In [69]:
def fill_na_values(data: pd.DataFrame, fill_na_val: dict) -> pd.DataFrame:
    """
    Заполнение пропусков заданными значениями
    :param data: датафрейм
    :param fill_na_val: словарь с названиями признаков и значением, которым нужно заполнить пропуки
    :return: датафрейм
    """
    return data.fillna(fill_na_val)

In [70]:
preprocessing['columns_fill_na']

{'price': -999}

In [71]:
# заполним пропуски
df = fill_na_values(df, preprocessing['columns_fill_na'])

In [72]:
def replace_model_mistakes(data: pd.DataFrame, replace_val: dict) -> pd.DataFrame:
    """
    Функция исправляет неточности в данных
    :param data: датафрейм с данными
    :param replace_val: словарь с признаками и значениями
    :return: датафрейм
    """
    return data.replace(replace_val)

In [115]:
df = replace_model_mistakes(df, preprocessing['replace_values'])

In [116]:
def replace_nokia_type(data: pd.DataFrame) -> pd.DataFrame:
    """
    Функция исправляет неточности в данных, заменяет
    :param data: датафрейм с данными
    :return: датафрейм с исправленными неточностями
    """
    data['cpe_type_cd'] = np.where((data['cpe_manufacturer_name'] == 'Nokia') &
                                   (data['cpe_model_name'] == '3 Dual'),
                                   'plain', data['cpe_type_cd'])
    return data

In [75]:
df = replace_nokia_type(df)

# Feature Generation

## Количество запросов в разные части дня

In [18]:
def get_data_part_day(data: pd.DataFrame) -> pd.DataFrame:
    """
    Функция аггрегирует данные по user_id и возвращает долю и кол-во визитов в разное время суток
    :param data: датафрейм с данными
    :return: аггрегированный датафрейм
    """
    df_part_day = pd.get_dummies(data[['user_id', 'part_of_day']])

    # кол-во визитов пользователя в разные части дня
    df_part_day = (df_part_day.groupby('user_id', as_index=False)
                   .agg({'part_of_day_day': 'sum',
                         'part_of_day_evening': 'sum',
                         'part_of_day_morning': 'sum',
                         'part_of_day_night': 'sum'}))

    # общее кол-во визитов пользователя
    df_part_day['sum_visits'] = (df_part_day.part_of_day_day
                                 + df_part_day.part_of_day_evening
                                 + df_part_day.part_of_day_morning
                                 + df_part_day.part_of_day_night)

    # доля визитов в разные части дня
    df_part_day['day_pct'] = df_part_day.part_of_day_day / df_part_day.sum_visits
    df_part_day['evening_pct'] = df_part_day.part_of_day_evening / df_part_day.sum_visits
    df_part_day['morning_pct'] = df_part_day.part_of_day_morning / df_part_day.sum_visits
    df_part_day['night_pct'] = df_part_day.part_of_day_night / df_part_day.sum_visits
    return df_part_day

In [19]:
df_part_day = get_data_part_day(df)
df_part_day.head(3)

,user_id,part_of_day_day,part_of_day_evening,part_of_day_morning,part_of_day_night,sum_visits,day_pct,evening_pct,morning_pct,night_pct
0,4,199,170,212,10,591,0.336717,0.287648,0.358714,0.016920
1,16,443,321,330,137,1231,0.359870,0.260764,0.268075,0.111292
2,18,566,114,411,34,1125,0.503111,0.101333,0.365333,0.030222


## Количество дней активности, среднее кол-во запросов в сутки

In [136]:
def get_data_days(data: pd.DataFrame) -> pd.DataFrame:
    """
    Функция аггрегирует данные по user_id и возвращает следующие признаки:
    - act_days - кол-во дат активности пользователя
    - request_cnt - кол-во запросов пользователя
    - avg_req_per_day - среднее кол-во запросов пользователя
    - period_days - кол-во дней между первым и последним визитом пользователя
    - act_days_pct - доля дней, когда пользователь совершал визит

    :param data: датафрейм с данными
    :return: аггрегированный датафрейм
    """
    # кол-во дней с визитами
    df_active_days = (data.groupby('user_id', as_index=False)
                      .agg({'date': 'nunique',
                            'request_cnt': 'sum'})
                      .rename(columns={'date': 'act_days'}))

    # среднее кол-во запросов в дни визита
    df_active_days['avg_req_per_day'] = (df_active_days.request_cnt /
                                         df_active_days.act_days)

    # первая и последняя дата визита
    df_dates_period = (data.groupby('user_id', as_index=False)
                       .agg({'date': ['max', 'min']}))

    # кол-во дней между первым и последним заходом
    df_dates_period['period_days'] = (df_dates_period['date'].iloc[:, 0] -
                                      df_dates_period['date'].iloc[:, 1])
    df_dates_period['period_days'] = df_dates_period.period_days.dt.days + 1
    df_dates_period = df_dates_period.drop('date', axis=1)

    df_dates_period.columns = df_dates_period.columns.droplevel(1)

    df_days = (df_active_days
               .merge(df_dates_period, on='user_id')
               )
    # доля дней, когда пользователь совершал визит
    df_days['act_days_pct'] = df_days.act_days / df_days.period_days

    return df_days

In [137]:
df_days = get_data_days(df)
df_days.head(3)

,user_id,act_days,request_cnt,avg_req_per_day,period_days,act_days_pct
0,4,20,777,38.850000,43,0.465116
1,16,64,2529,39.515625,74,0.864865
2,18,32,1950,60.937500,38,0.842105


## Цена устройства

In [117]:
def get_user_model_price(data: pd.DataFrame) -> pd.DataFrame:
    """
    Функция аггрегирует данные по user_id и возвращает следующие признаки:
    - cpe_type_cd - тип устройства
    - cpe_model_os_type - операционная система устройства
    - cpe_manufacturer_name - производитель устройства
    - price - цена устройства пользователя

    :param data: датафрейм с данными
    :return: аггрегированный датафрейм
    """
    df_model = data.groupby('user_id', as_index=False).agg({
        'cpe_type_cd': mode,
        'cpe_manufacturer_name': mode,
        'cpe_model_os_type': mode,
        'price': 'mean'
    })
    return df_model.fillna(-999)

In [118]:
df_model = get_user_model_price(df)
df_model.head(3)

,user_id,cpe_type_cd,cpe_manufacturer_name,cpe_model_os_type,price
0,4,smartphone,Huawei,Android,12990.0
1,16,smartphone,Samsung,Android,9583.0
2,18,smartphone,Samsung,Android,22887.0


## Регион и город

In [47]:
def get_user_city_cnt(data: pd.DataFrame) -> pd.DataFrame:
    """
    Функция аггрегирует данные по user_id и возвращает следующие признаки:
    - region_cnt - кол-во уникальных регионов, из которых был совершен визит
    - city_cnt - кол-во уникальных городов, из которых был совершен визит

    :param data: датафрейм с данными
    :return: аггрегированный датафрейм
    """
    df_city_cnt = data.groupby('user_id', as_index=False) \
        .agg({'region_name': 'nunique',
              'city_name': 'nunique'}) \
        .rename(columns={'region_name': 'region_cnt',
                         'city_name': 'city_cnt'})
    return df_city_cnt

In [48]:
df_city_cnt = get_user_city_cnt(df)
df_city_cnt.head(3)

,user_id,region_cnt,city_cnt
0,4,5,9
1,16,1,1
2,18,1,2


## URLs

In [49]:
def get_user_url_cnt(data: pd.DataFrame) -> pd.DataFrame:
    """
    Функция аггрегирует данные по user_id и возвращает следующие признаки:
    - url_host_cnt - кол-во уникальных ссылок, с которых был совершен визит

    :param data: датафрейм с данными
    :return: аггрегированный датафрейм
    """
    df_url_cnt = data.groupby('user_id', as_index=False) \
        .agg({'url_host': 'nunique'}) \
        .rename(columns={'url_host': 'url_host_cnt'})
    return df_url_cnt

In [50]:
df_url = get_user_url_cnt(df)
df_url.head(3)

,user_id,url_host_cnt
0,4,108
1,16,50
2,18,141


# Итоговый датафрейм

Теперь проделаем те же действия для каждого файла с данными и объединим все в общий датафрейм

In [53]:
config['preprocessing']['agg_data_path']

'../data/processed/agg_data.csv'

In [57]:
config['preprocessing']['raw_data_extension']

'.parquet'

In [125]:
# название папки с файлами
DATA_FOLD = config['preprocessing']['raw_data_path']

df_final = pd.DataFrame()
for file in tqdm.tqdm_notebook(os.listdir(DATA_FOLD)):
    if file.endswith(config['preprocessing']['raw_data_extension']):
        temp_df = pd.read_parquet(f'{DATA_FOLD}/{file}', engine='fastparquet')
        # меняем типы данных
        temp_df = change_cols_type(temp_df, preprocessing['change_col_types'])
        # заполним пропуски
        temp_df = fill_na_values(temp_df, preprocessing['columns_fill_na'])
        # заменяем неточный значения
        temp_df = replace_model_mistakes(temp_df, preprocessing['replace_values'])
        # заменяем неточности типа устройства nokia
        temp_df = replace_nokia_type(temp_df)
        

        # получаем аггрегированные данные
        # кол-во визитов пользователя в разное время суток
        data_part_day = get_data_part_day(temp_df)
        # кол-во активных дней и среднее кол-во запросов в сутки
        data_days = get_data_days(temp_df)
        # данные об устройстве
        data_user_model = get_user_model_price(temp_df)
        # кол-во регионов и городов
        data_city_cnt = get_user_city_cnt(temp_df)
        # кол-во уникальных url
        data_url_cnt = get_user_url_cnt(temp_df)

        # объединяем все в один датафрейм
        temp_df_agg = (data_part_day
                       .merge(data_days, how='left', on='user_id')
                       .merge(data_user_model, how='left', on='user_id')
                       .merge(data_city_cnt, how='left', on='user_id')
                       .merge(data_url_cnt, how='left', on='user_id'))

        # добавляем аггрегированные данные в итоговый датафрейм
        df_final = pd.concat([df_final, temp_df_agg])

  0%|          | 0/11 [00:00<?, ?it/s]

In [126]:
df_final.head()

,user_id,part_of_day_day,part_of_day_evening,part_of_day_morning,part_of_day_night,sum_visits,day_pct,evening_pct,morning_pct,night_pct,...,period_days,requests_std,act_days_pct,cpe_type_cd,cpe_manufacturer_name,cpe_model_os_type,price,region_cnt,city_cnt,url_host_cnt
0,13,467,447,411,226,1551,0.301096,0.288201,0.264990,0.145712,...,116,1.618662,0.974138,smartphone,BQ Devices Limited,Android,-999.000000,1,1,83
1,41,7,13,9,9,38,0.184211,0.342105,0.236842,0.236842,...,4,1.562360,1.000000,smartphone,Xiaomi,Android,12343.000000,1,3,15
2,69,457,346,314,30,1147,0.398431,0.301656,0.273758,0.026155,...,60,1.331956,0.933333,smartphone,Apple,iOS,16657.000000,1,1,91
3,71,49,57,12,7,125,0.392000,0.456000,0.096000,0.056000,...,8,0.865094,1.000000,smartphone,Apple,iOS,38037.000000,2,2,35
4,85,838,1067,602,333,2840,0.295070,0.375704,0.211972,0.117254,...,41,1.144101,0.926829,smartphone,Apple,iOS,55809.003906,2,6,174


In [127]:
df_final.shape

(415317, 23)

In [128]:
# проверим, что файлы не пересекались по user_id
(df_final.groupby('user_id', as_index=False)
         .agg({'city_cnt': 'count'})
         .query('city_cnt > 1'))

,user_id,city_cnt


In [85]:
config['preprocessing']['agg_data_path']

'../data/processed/agg_data.csv'

In [141]:
# сохраняем итоговый аггрегированный датафрейм
df_final.to_csv(config['preprocessing']['agg_data_path'], index=False)

In [131]:
# функция для сохранения уникальных значений в наших данных
# для дальнейшего их отображения во frontend части
def save_unique_train_data(data: pd.DataFrame, 
                           columns_save_unique: list,
                           columns_save_min_max: list,
                           unique_values_path: str) -> None:
    """
    Сохранение словаря с признаками и уникальными значениями
    :param data: датасет
    :param columns_save_unique: признаки, для которых нужно сохранить уникальные значения
    :param columns_save_min_max: признаки, для которых нужно сохранить только мин и макс
    :param unique_values_path: путь до файла со словарем
    :return: None
    """
    unique_df = data[columns_save_unique]
    min_max_df = data[columns_save_min_max]

    # создаем словарь с уникальными значениями для вывода в UI
    dict_unique = {
        key: unique_df[key].unique().tolist()
        for key in unique_df.columns
    }

    # создаем словарь со значениями min, max
    dict_min_max = {
        key: [
            math.floor(min_max_df[key].min())
            if min_max_df[key].min() > 0 else 0,
            math.ceil(min_max_df[key].max())
        ]
        for key in min_max_df.columns
    }

    # объединяем словари
    dict_unique.update(dict_min_max)

    with open(unique_values_path, "w") as file:
        json.dump(dict_unique, file)

In [144]:
# сохраним уникальные значения
save_unique_train_data(
    data=df_final,
    columns_save_unique=preprocessing['columns_save_unique'],
    columns_save_min_max=preprocessing['columns_save_min_max'],
    unique_values_path=preprocessing['unique_values_path'])